# Capstone Project - Battle Of Neighborhoods

## This is the capstone project for the IBM Data Science Professional Certificate
### Open new Mexican Restaurant in London, United Kingdom

#### Introduction/Business Problem:

In [1]:
# 1. A description of the problem and a discussion of the background

London is the capital and largest city of England and the United Kingdom. It is the third-most populous in Europe. London has a diverse range of people and cultures, and more than 300 languages are spoken in the region.  It offers many business opportunities, like openning new restaurants so is competitive. Careful consideration of analysis for the business expansion is so much important as it is directly proportional to the cost of the business. The analysis from this report help for the new businesses strategically target the market and help in a high return on investment so is low risk. <br> Opening a new restaurant can be daunting. There are so many things to consider from choosing the right location to finding the financing to selecting the right name. As the location is one of the main factors in opening a new restaurant we will focus on the restaurant business around the London area in this report. The first step in opening a new restaurant is deciding what type of restaurant it is going to be? Is it a high-end fine dining restaurant or casual style dinner or a specific type of cuisine in mind such as French, Italian or Indian. Choosing a restaurant located in a busy area with plenty of foot traffic, or is there enough parking or are there any other restaurants opened or closed in the same spot?
We will focus on the types of different restaurants which are opened or closed in a particular place and then decide if it is a good place to open a new restaurant based on the popular cuisine around that place.

In [2]:
# 2. A description of the data and how it will be used to solve the problem

#### Target Audience:

The objective of my project is to recommend which neighborhood of London is a good choice for a new Mexican Restaurant business to open.

#### Methodology - How will I use the data to solve the described Business Problem:

- I will convert address data into their equivalent latitude and longitude values.
- For London neighborhood data, I will use the resource from wikipedia page , https://en.wikipedia.org/wiki/List_of_areas_of_London
- Use the Foursquare API to explore London neighborhoods and to get venues in neighborhoods.
- Use the Foursquare API to get venue ratings and likes in neighborhoods.
- Use the k-means clustering to complete clustering task
- Use the Folium library to visualize the neighborhoods, venues , clusters in London